In [242]:
import warnings

# 오류 경고 무시하기
warnings.filterwarnings(action='ignore')

# 1. 데이터 불러오기

In [243]:
import pandas as pd
import os

pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_columns', None)

PATH = './data/'

file_list= os.listdir(PATH)
csv_list = list()

for file in file_list:
    if file.split(".")[-1] == 'csv':
        csv_list.append(file)
data = ['living_popul', 'resident_popul', 'area', 'store', 'working_popul', 'facilitie', 'estimated_sales']

for name, file in zip(data, csv_list):
    globals()[name] = pd.read_csv(PATH + file, encoding = "ansi")

store = store.drop("Unnamed: 0", axis = 1)
estimated_sales = estimated_sales.drop("Unnamed: 0", axis = 1)

# 2. 데이터 전처리

## 2-0. 시점 통일 / 관광특구 제거

In [244]:
# 시점 통일
years = [20223, 20224, 20231, 20232]

living_popul = living_popul[living_popul['기준_년분기_코드'].isin(years)]
resident_popul = resident_popul[resident_popul['기준_년분기_코드'].isin(years)]
store = store[store['기준_년분기_코드'].isin(years)]
working_popul = working_popul[working_popul['기준_년분기_코드'].isin(years)]
facilitie = facilitie[facilitie['기준_년분기_코드'].isin(years)]
estimated_sales = estimated_sales[estimated_sales['기준_년분기_코드'].isin(years)]

# 관광특구 제거
living_popul = living_popul[living_popul['상권_구분_코드_명']!='관광특구']
resident_popul = resident_popul[resident_popul['상권_구분_코드_명']!='관광특구']
store = store[store['상권_구분_코드_명']!='관광특구']
working_popul = working_popul[working_popul['상권_구분_코드_명']!='관광특구']
facilitie = facilitie[facilitie['상권_구분_코드_명']!='관광특구']
estimated_sales = estimated_sales[estimated_sales['상권_구분_코드_명']!='관광특구']

## 2-1. 길단위인구 전처리

### 2-1-1 변수 처리

In [245]:
# 필요힌 변수 선택
living_popul = living_popul[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '연령대_10_유동인구_수',
                            '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수',
                            '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수']]

In [246]:
living_popul = living_popul.rename(columns = {'연령대_10_유동인구_수' : '10대',
                                             '연령대_20_유동인구_수' : '20대',
                                             '연령대_30_유동인구_수' : '30대',
                                             '연령대_40_유동인구_수' : '40대',
                                             '연령대_50_유동인구_수' : '50대',
                                             '연령대_60_이상_유동인구_수' : '60대 이상'})

In [247]:
living_popul = pd.melt(living_popul, id_vars = ['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명'],
                      var_name = '연령', value_name = '유동인구수')

### 2-1-3. 분기별 데이터 연도 기준 변환

In [248]:
living_popul_year = living_popul.drop('기준_년분기_코드', axis = 1)\
                                .groupby(['상권_구분_코드_명','상권_코드_명', '연령'],as_index = False)\
                                .mean()

In [249]:
living_popul_year

,상권_구분_코드_명,상권_코드_명,연령,유동인구수
0,골목상권,4.19민주묘지역 2번,10대,128381.000
1,골목상권,4.19민주묘지역 2번,20대,163223.250
2,골목상권,4.19민주묘지역 2번,30대,107811.500
3,골목상권,4.19민주묘지역 2번,40대,113781.750
4,골목상권,4.19민주묘지역 2번,50대,121303.500
...,...,...,...,...
9859,전통시장,흑석시장,20대,15602.750
9860,전통시장,흑석시장,30대,12310.750
9861,전통시장,흑석시장,40대,13962.250
9862,전통시장,흑석시장,50대,12309.250


## 2-2. 상주인구 전처리

### 2-2-1. 변수 처리

In [250]:
# 필요한 변수 선택
resident_popul = resident_popul[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명',
                                '연령대_10_상주인구_수', '연령대_20_상주인구_수', '연령대_30_상주인구_수', '연령대_40_상주인구_수',
                                '연령대_50_상주인구_수', '연령대_60_이상_상주인구_수']]

In [251]:
resident_popul = resident_popul.rename(columns = {'연령대_10_상주인구_수' : '10대',
                                             '연령대_20_상주인구_수' : '20대',
                                             '연령대_30_상주인구_수' : '30대',
                                             '연령대_40_상주인구_수' : '40대',
                                             '연령대_50_상주인구_수' : '50대',
                                             '연령대_60_이상_상주인구_수' : '60대 이상'})

In [252]:
resident_popul = pd.melt(resident_popul, id_vars = ['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명'],
                      var_name = '연령', value_name = '상주인구수')

### 2-2-3. 분기별 데이터 연도 기준 변환

In [253]:
resident_popul_year = resident_popul[resident_popul['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드'],axis=1)

In [254]:
resident_popul_year

,상권_구분_코드_명,상권_코드_명,연령,상주인구수
4854,골목상권,이북5도청사,10대,116
4855,골목상권,독립문역 1번,10대,52
4856,골목상권,세검정초등학교,10대,331
4857,골목상권,대신고등학교,10대,51
4858,골목상권,세검정,10대,243
...,...,...,...,...
38923,전통시장,마천시장,60대 이상,55
38924,전통시장,마천중앙시장,60대 이상,438
38925,전통시장,명일전통시장,60대 이상,551
38926,전통시장,고덕 골목형상점가,60대 이상,53


## 2-3. 점포 전처리

### 2-3-1. 변수 처리

In [255]:
# 필요한 변수 선택
store = store[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '서비스_업종_코드_명',
              '유사_업종_점포_수']]

### 2-3-2. 분기별 데이터 연도 기준 변환

In [256]:
store_year = store[store['기준_년분기_코드'] == 20232].drop('기준_년분기_코드', axis = 1)\
                                                    .groupby(['상권_구분_코드_명', '상권_코드_명', '서비스_업종_코드_명'], as_index = False)\
                                                    .sum()

In [257]:
store_year

,상권_구분_코드_명,상권_코드_명,서비스_업종_코드_명,유사_업종_점포_수
0,골목상권,4.19민주묘지역 2번,건축물청소,3
1,골목상권,4.19민주묘지역 2번,네일숍,1
2,골목상권,4.19민주묘지역 2번,노래방,1
3,골목상권,4.19민주묘지역 2번,당구장,1
4,골목상권,4.19민주묘지역 2번,미용실,3
...,...,...,...,...
75584,전통시장,흑석시장,인테리어,2
75585,전통시장,흑석시장,일반의류,3
75586,전통시장,흑석시장,청과상,2
75587,전통시장,흑석시장,한식음식점,11


## 2-4. 직장인구 전처리

### 2-4-1. 변수 처리

In [258]:
working_popul = working_popul[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명',
                              '연령대_10_직장_인구_수', '연령대_20_직장_인구_수', '연령대_30_직장_인구_수',
                              '연령대_40_직장_인구_수', '연령대_50_직장_인구_수', '연령대_60_이상_직장_인구_수']]

In [259]:
working_popul = working_popul.rename(columns = {'연령대_10_직장_인구_수' : '10대',
                                             '연령대_20_직장_인구_수' : '20대',
                                             '연령대_30_직장_인구_수' : '30대',
                                             '연령대_40_직장_인구_수' : '40대',
                                             '연령대_50_직장_인구_수' : '50대',
                                             '연령대_60_이상_직장_인구_수' : '60대 이상'})

In [260]:
working_popul = pd.melt(working_popul, id_vars = ['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명'],
                      var_name = '연령', value_name = '직장인구수')

### 2-4-3. 분기별 데이터 연도 기준 변환

In [261]:
working_popul_year = working_popul[working_popul['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드'],axis=1)

In [262]:
working_popul_year

,상권_구분_코드_명,상권_코드_명,연령,직장인구수
4830,발달상권,북촌(안국역),10대,8
4831,골목상권,중앙고등학교,10대,0
4832,골목상권,부암동주민센터,10대,0
4833,골목상권,성곡미술관,10대,0
4834,발달상권,서촌(경복궁역),10대,37
...,...,...,...,...
38803,골목상권,한영중고,60대 이상,50
38804,골목상권,천호역 6번,60대 이상,77
38805,골목상권,둔촌동역 4번,60대 이상,23
38806,골목상권,천호초등학교,60대 이상,27


## 2-5. 집객시설 전처리

### 2-5-1. 변수 처리

In [263]:
# 파생변수 생성
facilitie['교통시설수'] = facilitie['버스_정거장_수'] + facilitie['지하철_역_수']
 
facilitie['집객시설수'] = facilitie['집객시설_수']-facilitie['교통시설수']

# 필요한 변수 선택
facilitie = facilitie[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '집객시설수', '교통시설수' ]]

In [264]:
# 결측값 0으로 대체
facilitie = facilitie.fillna(0)

### 2-5-3. 분기별 데이터 연도 기준 변환

In [265]:
facilitie_year = facilitie[facilitie['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드'], axis = 1)

In [266]:
facilitie_year

,상권_구분_코드_명,상권_코드_명,집객시설수,교통시설수
0,골목상권,동묘앞역 2번,0.000,0.000
1,골목상권,목동로데오거리,0.000,0.000
3,골목상권,청운초등학교,0.000,0.000
5,골목상권,성곡미술관,0.000,0.000
7,골목상권,경복고등학교,0.000,0.000
...,...,...,...,...
8715,골목상권,문래소공원,0.000,0.000
11885,골목상권,서초소방서,0.000,0.000
12137,골목상권,언주역 3번,0.000,0.000
17142,전통시장,신설종합시장,0.000,0.000


## 2-6. 추정매출 전처리

### 2-6-1. 변수 처리

In [267]:
# 필요한 변수 선택
estimated_sales = estimated_sales[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', 
                                  '서비스_업종_코드_명', '연령대_10_매출_금액',
                                  '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액',
                                  '연령대_50_매출_금액', '연령대_60_이상_매출_금액']]

In [268]:
estimated_sales = estimated_sales.rename(columns = {'연령대_10_매출_금액' : '10대',
                                             '연령대_20_매출_금액' : '20대',
                                             '연령대_30_매출_금액' : '30대',
                                             '연령대_40_매출_금액' : '40대',
                                             '연령대_50_매출_금액' : '50대',
                                             '연령대_60_이상_매출_금액' : '60대 이상'})

In [269]:
estimated_sales = pd.melt(estimated_sales, id_vars = ['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '서비스_업종_코드_명'],
                      var_name = '연령', value_name = '매출금액')

### 2-6-3. 분기별 데이터 연도 기준 변환

In [270]:
estimated_sales_year = estimated_sales.drop(columns = ['기준_년분기_코드'])\
                                      .groupby(['상권_구분_코드_명', '상권_코드_명', '서비스_업종_코드_명', '연령'])\
                                      .mean()\
                                      .reset_index()

In [271]:
estimated_sales_year

,상권_구분_코드_명,상권_코드_명,서비스_업종_코드_명,연령,매출금액
0,골목상권,4.19민주묘지역 2번,미용실,10대,136012.500
1,골목상권,4.19민주묘지역 2번,미용실,20대,3947675.500
2,골목상권,4.19민주묘지역 2번,미용실,30대,7595096.000
3,골목상권,4.19민주묘지역 2번,미용실,40대,13009877.000
4,골목상권,4.19민주묘지역 2번,미용실,50대,5901769.750
...,...,...,...,...,...
132031,전통시장,흑석시장,한식음식점,20대,21435286.000
132032,전통시장,흑석시장,한식음식점,30대,21683765.250
132033,전통시장,흑석시장,한식음식점,40대,37320887.000
132034,전통시장,흑석시장,한식음식점,50대,45442558.250


## 2-7. 영역 전처리

### 2-7-1. 변수처리

In [272]:
area = area[['상권_구분_코드_명', '상권_코드_명', '자치구_코드_명']]

## 2-7. 데이터 병합

In [273]:
preprocessed_df = pd.merge(living_popul_year, resident_popul_year)
preprocessed_df = pd.merge(preprocessed_df, store_year)
preprocessed_df = pd.merge(preprocessed_df, working_popul_year)
preprocessed_df = pd.merge(preprocessed_df, facilitie_year)
preprocessed_df = pd.merge(preprocessed_df, estimated_sales_year)
preprocessed_df = pd.merge(preprocessed_df, area)

In [274]:
preprocessed_df

,상권_구분_코드_명,상권_코드_명,연령,유동인구수,상주인구수,서비스_업종_코드_명,유사_업종_점포_수,직장인구수,집객시설수,교통시설수,매출금액,자치구_코드_명
0,골목상권,4.19민주묘지역 2번,10대,128381.000,135,미용실,3,0,0.000,0.000,136012.500,강북구
1,골목상권,4.19민주묘지역 2번,10대,128381.000,135,분식전문점,3,0,0.000,0.000,2272384.000,강북구
2,골목상권,4.19민주묘지역 2번,10대,128381.000,135,일식음식점,3,0,0.000,0.000,561557.250,강북구
3,골목상권,4.19민주묘지역 2번,10대,128381.000,135,치킨전문점,3,0,0.000,0.000,208308.750,강북구
4,골목상권,4.19민주묘지역 2번,10대,128381.000,135,커피-음료,4,0,0.000,0.000,6104389.500,강북구
...,...,...,...,...,...,...,...,...,...,...,...,...
130051,전통시장,후암시장,60대 이상,33814.250,305,일반교습학원,3,19,0.000,0.000,0.000,용산구
130052,전통시장,후암시장,60대 이상,33814.250,305,일반의류,6,19,0.000,0.000,0.000,용산구
130053,전통시장,후암시장,60대 이상,33814.250,305,컴퓨터및주변장치판매,3,19,0.000,0.000,0.000,용산구
130054,전통시장,후암시장,60대 이상,33814.250,305,편의점,3,19,0.000,0.000,20101666.000,용산구
